In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import json
import pickle
import argparse
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from process_data import save_pickle, load_pickle, load_processed_data, load_glove_weights, to_var, make_word_vector, add_padding
from word_embedding import WordEmbedding
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import sys
from jnet import JNet

In [2]:
sys.argv = ['a.py']

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=8, help='input batch size')
parser.add_argument('--embd_size', type=int, default=100, help='word embedding size')
args = parser.parse_args()

In [3]:
# from process_data import save_pickle, load_pickle, load_task, load_glove_weights, to_var, make_word_vector, make_one_hot
# train_data, train_ctx_maxlen = load_task('./dataset/train-v1.1.json')
# save_pickle(train_data, './pickles/train_data.pickle')
# train_data = load_pickle('./pickles/train_data.pickle')
# dev_data, dev_ctx_maxlen = load_task('./dataset/dev-v1.1.json')
# save_pickle(dev_data, './pickles/dev_data.pickle')
# dev_data = load_pickle('./pickles/dev_data.pickle')
# print('N train', len(train_data))
# print('N dev', len(dev_data))
# data = train_data+dev_data
# ctx_maxlen = max(train_ctx_maxlen, dev_ctx_maxlen)
# ctx_maxlen = 4063 #TODO
# args.ctx_maxlen = ctx_maxlen
# print('context char-level maxlen:', ctx_maxlen)
train_data = load_processed_data('./dataset/train.txt')
data = train_data
vocab = set()
for ctx, query, _ in data:
    vocab |= set(ctx + query)
    
vocab = list(sorted(vocab)) + ['PAD']
w2i = dict((w, i) for i, w in enumerate(vocab, 1))
i2w = dict((i, w) for i, w in enumerate(vocab, 1))
args.vocab_size = len(vocab)
print('vocab size', len(vocab))

ctx_token_maxlen = max([len(c) for c, _, _ in data])
query_token_maxlen = max([len(q) for _, q, _ in data])
print('ctx_token_maxlen:', ctx_token_maxlen)
print('query_token_maxlen:', query_token_maxlen)
args.ctx_token_maxlen = ctx_token_maxlen
args.query_token_maxlen = query_token_maxlen
args.answer_token_len = 1 # 2 TODO

glove_embd = load_pickle('./pickles/glove_embd.pickle')
# glove_embd = torch.from_numpy(load_glove_weights('./dataset', args.embd_size, len(vocab), w2i)).type(torch.FloatTensor)
# save_pickle(glove_embd, './pickles/glove_embd.pickle')
args.pre_embd = glove_embd

vocab size 34762
ctx_token_maxlen: 100
query_token_maxlen: 40
load ./pickles/glove_embd.pickle


In [6]:
data[10]

(['the',
  'television',
  'station',
  ',',
  'ndtv',
  ',',
  'grew',
  'from',
  'one',
  'show',
  'in',
  '2002',
  'to',
  'a',
  'full',
  '24-hour',
  'channel',
  'with',
  'original',
  'programming',
  'by',
  'september',
  '2006',
  '.',
  'wsnd-fm',
  'serves',
  'the',
  'student',
  'body',
  'and',
  'larger',
  'south',
  'bend',
  'community',
  'at',
  '88.9',
  'fm',
  ',',
  'offering',
  'students',
  'a',
  'chance',
  'to',
  'become',
  'involved',
  'in',
  'bringing',
  'classical',
  'music',
  ',',
  'fine',
  'arts',
  'and',
  'educational',
  'programming',
  ',',
  'and',
  'alternative',
  'rock',
  'to',
  'the',
  'airwaves',
  '.',
  'another',
  'radio',
  'station',
  ',',
  'wvfi',
  ',',
  'began',
  'as',
  'a',
  'partner',
  'of',
  'wsnd-fm',
  '.',
  'more',
  'recently',
  ',',
  'however',
  ',',
  'wvfi',
  'has',
  'been',
  'airing',
  'independently',
  'and',
  'is',
  'streamed',
  'on',
  'the',
  'internet',
  '.'],
 ['which',
  

In [ ]:
from jnet import JNet
def train(data, model, optimizer, loss_fn, n_epoch=5, batch_size=32):
    for epoch in range(n_epoch):
        print('---Epoch', epoch)
        for i in tqdm(range(0, len(data)-batch_size, batch_size)): # TODO shuffle, last elms
            batch_data = data[i:i+batch_size]
            c = [d[0] for d in batch_data]
            q = [d[1] for d in batch_data]
            context_var = make_word_vector(c, w2i, ctx_token_maxlen)
            query_var = make_word_vector(q, w2i, query_token_maxlen)
#             labels = [[d[4][0], d[5][0]] for d in batch_data]
            labels = [d[2][0] for d in batch_data]
            labels = to_var(torch.LongTensor(labels))
            outs = model(context_var, query_var)
            outs = outs.view(-1, ctx_token_maxlen) #(B*M, L)
#             print('preds', torch.max(outs, 1)[1])
            labels = labels.view(-1) # (B*M)
#             print('labels', labels)
            loss = loss_fn(outs, labels)
            if i % (batch_size*10) == 0:
                print(loss.data[0])
            model.zero_grad()
            loss.backward()
            optimizer.step()
#             break
        
        _, preds = torch.max(outs, 1)
        ct = 0
        for pred, label in zip(preds, labels):
            if pred.data[0] == label.data[0]: 
                ct+=1
        print('Acc', ct, '/', len(preds))
        break
        
    
model = JNet(args)
if torch.cuda.is_available():
    model.cuda()

# print(model)
# for p in model.parameters():
#     print(p)
loss_fn = nn.NLLLoss()
# optimizer = torch.optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

# optimizer = torch.optim.Adadelta(model.parameters(), lr=0.5, weight_decay=0.999)
train(train_data, model, optimizer, loss_fn)
print('fin')

In [ ]:
def lower_list(str_list):
    return [str_var.lower() for str_var in str_list]
def preprocess(fpath_read, fpath_write):
    count = 0
    fpr = open(fpath_read, 'r')
    body = fpr.read()
    js = json.loads(body)
    fpw = open(fpath_write, 'w')
    for c in js["data"]:
        print(c['title'])
        for p in c["paragraphs"]:
            context = p["context"].split(' ')
            context_char = list(p["context"])
            context_pos = {}
            for qa in p["qas"]:

                question = word_tokenize(qa["question"])

                for a in qa['answers']:
                    answer = a['text'].strip()
                    answer_start = int(a['answer_start'])

                #add '.' here, just because NLTK is not good enough in some cases
                answer_words = word_tokenize(answer+'.')
                if answer_words[-1] == '.':
                    answer_words = answer_words[:-1]
                else:
                    answer_words = lower_list(word_tokenize(answer))

                prev_context_words = lower_list(word_tokenize( p["context"][0:answer_start ] ))
                left_context_words = lower_list(word_tokenize( p["context"][answer_start:] ))
                answer_reproduce = []
                for i in range(len(answer_words)):
                    if i < len(left_context_words):
                        w = left_context_words[i]
                        answer_reproduce.append(w)
                join_a = ' '.join(answer_words)
                join_ar = ' '.join(answer_reproduce)

                #if not ((join_ar in join_a) or (join_a in join_ar)):
                if join_a != join_ar:
                    #print join_ar
                    #print join_a
                    #print 'answer:'+answer
                    count += 1

                fpw.write(' '.join(prev_context_words+left_context_words)+'\t')
                fpw.write(' '.join(question)+'\t')
                #fpw.write(join_a+'\t')

                pos_list = []
                for i in range(len(answer_words)):
                    if i < len(left_context_words):
                        pos_list.append(str(len(prev_context_words)+i+1))
                if len(pos_list) == 0:
                    print (join_ar)
                    print (join_a)
                    print ('answer:'+answer)
                assert(len(pos_list) > 0)
                fpw.write(' '.join(pos_list)+'\n')

    fpw.close()
print ('SQuAD preprossing finished!')
preprocess()

In [10]:
a = torch.Tensor([[
    [1, 2, 3],
    [4, 5, 6]
]])
print('a', a.size())
b = torch.Tensor([[
    [1, 1, 1]
]])
print('b', b.size())

c = a+b
print(c)

a torch.Size([1, 2, 3])
b torch.Size([1, 1, 3])

(0 ,.,.) = 
  2  3  4
  5  6  7
[torch.FloatTensor of size 1x2x3]



In [22]:
a=torch.Tensor([[
    [0.2, 0.5, 0.3],
    [0, 0, 1]
]])
print(a)
print(a.view(-1, 3))
print(F.softmax(a))
print(F.softmax(a.view(-1, 3)).view(1, 2, 3))



(0 ,.,.) = 
  0.2000  0.5000  0.3000
  0.0000  0.0000  1.0000
[torch.FloatTensor of size 1x2x3]


 0.2000  0.5000  0.3000
 0.0000  0.0000  1.0000
[torch.FloatTensor of size 2x3]

Variable containing:
(0 ,.,.) = 
  1  1  1
  1  1  1
[torch.FloatTensor of size 1x2x3]

Variable containing:
(0 ,.,.) = 
  0.2894  0.3907  0.3199
  0.2119  0.2119  0.5761
[torch.FloatTensor of size 1x2x3]

